In [1]:
%pip install transformer_lens
%pip install datasets
%pip install plotly
%pip install sae-lens
%pip install openai
%pip install tqdm


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 

In [2]:
import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
import tqdm

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

# from steering.eval_utils import evaluate_completions
from steering.utils import get_activation_steering, get_sae_diff_steering, remove_sae_feats, text_to_sae_feats, top_activations
from steering.patch import generate, get_loss

import plotly.express as px

torch.set_grad_enabled(False)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HookedTransformer.from_pretrained('gpt2-small', device=device)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
hp_6 = tutils.get_act_name("resid_pre", 6)
sae_6 = get_gpt2_res_jb_saes(hp_6)[0][hp_6]
sae_6 = sae_6.to(model.W_E.device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


In [5]:
from enum import Enum
from tqdm import tqdm
import random

# Direction for steering vector
class STEER(Enum):
    NEGATIVE = -1
    POSITIVE = 1

In [6]:
@torch.no_grad()
def get_activation_steering(
        model: HookedTransformer,
        hook_point: str,
        prompts,
    ):
    """
    Computes the activations for all prompts

    Returns:
        [torch.Tensor]: The activations for each prompt

    Example:
        prompts = ["anger", "calm"]
        hp = get_act_name("resid_pre", 4)  # blocks.4.hook_resid_pre
        steering = get_activation_steering(model, hp, prompts)
        print(steering.shape)  # torch.Size([2, 3, 768])
    """

    tokens = model.to_tokens(prompts)
    _, acts = model.run_with_cache(tokens, names_filter=hook_point)
    return acts[hook_point]

In [7]:
def patch_resid(resid, hook, steering_vectors, steering_directions, c=1, pos=0):
    if steering_vectors == None or steering_directions == None:
      return resid
    for i, steering_vector in enumerate(steering_vectors):
      # for pos in range(steering_vector.shape[0]):
      #   resid[0, pos] += steering_vector[pos] * steering_directions[i].value * c
      resid[0,0:steering_vector.shape[0]] += steering_vector * steering_directions[i].value * c
    return resid

def generate(
        model: HookedTransformer,
        hook_point: str,
        prompt = "",
        steering_vectors=None,
        steering_directions=None,
        scale=1,
        n_samples=5,
        max_length=20,
    ):
    samples = []
    for sample in tqdm(range(n_samples)):
      sample_text = prompt
      for i in range(max_length):

        logits = model.run_with_hooks(sample_text, 
            fwd_hooks=[(
              hook_point,
              partial(patch_resid, c=scale, steering_vectors=steering_vectors, steering_directions=steering_directions)
            )],
            prepend_bos=True)

        sample_text += model.tokenizer.decode(random.choice(logits[0][-1].topk(11).indices))
      samples.append(sample_text)
    return samples

In [8]:
text = "Thanks for asking about that! I moved to Berkeley, CA because"

steering_directions = [STEER.POSITIVE, STEER.NEGATIVE]
prompts = ["Dragons live in Berkeley", "People live in Berkeley"]
activation_vectors = get_activation_steering(model, hp_6, prompts)

assert(len(prompts) == len(steering_directions))

generate(model, hp_6, prompt=text, steering_vectors=activation_vectors, steering_directions=steering_directions, scale=4)

100%|██████████| 5/5 [00:14<00:00,  2.84s/it]


["Thanks for asking about that! I moved to Berkeley, CA because this isn´ts so easy for us. My parents had moved out a year before but there's",
 'Thanks for asking about that! I moved to Berkeley, CA because of what I was taught by your teachers in college so there I thought there could have be some good',
 "Thanks for asking about that! I moved to Berkeley, CA because my family's favorite mountain range (Mount Rose of Alameda) wasn�ts so there isn��",
 'Thanks for asking about that! I moved to Berkeley, CA because of our awesome city." ―Derek [To Daren Darian] The rest... I don',
 'Thanks for asking about that! I moved to Berkeley, CA because of their new library (which also includes many great things) with an extensive library for bookshelvers']

In [9]:
generate(model, hp_6, prompt=text)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:16<00:00,  3.36s/it]


["Thanks for asking about that! I moved to Berkeley, CA because it's awesome here, with so much great stuff! I've also gotten into astronomy, physics of",
 'Thanks for asking about that! I moved to Berkeley, CA because my husband got sick from cancer a couple months into living as he has done and now works in retail',
 'Thanks for asking about that! I moved to Berkeley, CA because that would help get a decent job. But I have also seen a good chunk more in my local',
 "Thanks for asking about that! I moved to Berkeley, CA because they've made me an engineer to go work out.\nI started in engineering when the kids got",
 "Thanks for asking about that! I moved to Berkeley, CA because a friend said there might still happen an election if you wanted to run and there'd definitely come that"]

In [10]:
text = "I hate you because"

steering_directions = [STEER.POSITIVE, STEER.NEGATIVE]
prompts = ["Love", "Hate"]
activation_vectors = get_activation_steering(model, hp_6, prompts)

assert(len(prompts) == len(steering_directions))

generate(model, hp_6, prompt=text, steering_vectors=activation_vectors, steering_directions=steering_directions, scale=10)

100%|██████████| 5/5 [00:13<00:00,  2.64s/it]


['I hate you because he\'s so amazing!" She is so incredibly beautiful with an almost incredible level for her, so if',
 "I hate you because he's not the one we'll ever need, and the world may find out and love us for",
 'I hate you because my husband (a big, bad ass man with an incredible heart, a very hard work in her',
 "I hate you because she's in this city at least one time that they will give the right people their best efforts.",
 'I hate you because you deserve it and if a young woman is in a room of the best-daring men who']